In [ ]:
from typing_extensions import TypedDict
class State(TypedDict):
    graph_state: str

In [ ]:
def first_node(state:State) :
    print("My first node is called")
    return {"graph_state": state["graph_state"]+"I am playing "}

def second_node(state:State) :
    print("My second node is called")
    return {"graph_state": state["graph_state"]+"cricket"}

def third_node(state:State) :
    print("My third node is called")
    return {"graph_state": state["graph_state"]+"basketball"}


In [ ]:
import random
from typing import Literal


## This function is called to decide which node to play next which will be LLM in future
def decide_play(state)->Literal['second_node', 'third_node']:
    graph_state = state["graph_state"]
    if random.random() > 0.5:
        return 'third_node'
    return 'second_node'


## GRAPH

In [ ]:
from IPython.display import Image, display
from langgraph.graph import StateGraph, START, END

#BUILD THE GRAPH
builder = StateGraph(State)
builder.add_node("first_node",first_node)
builder.add_node("second_node",second_node)
builder.add_node("third_node",third_node)

## LOGIC
builder.add_edge(START,"first_node")
builder.add_conditional_edges("first_node",decide_play)
builder.add_edge("second_node",END)
builder.add_edge("third_node",END)

# COMPILE AND DISPLY
graph = builder.compile()

display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
graph.invoke({"graph_state":"Hi , I am Kinnary. "})

## START WITH BASIC CHATBOT with LANGGRAPH

In [ ]:
from typing_extensions import TypedDict
from typing import Annotated
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
import os
import dotenv
dotenv.load_dotenv()
os.environ['GROQ_API_KEY'] = os.environ['GROQ_API_KEY']


In [ ]:
class State(TypedDict):
    messages:Annotated[list[str],add_messages]


In [ ]:
from langchain_groq import ChatGroq
llm=ChatGroq(model="gemma2-9b-it")
llm

def chatbot(state:State):
    return {"messages":[llm.invoke(state["messages"])]}

In [ ]:
graph_builder = StateGraph(State)
graph_builder.add_node("chatbot",chatbot)
graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)
graph = graph_builder.compile()

In [ ]:
from IPython.display import Image, display
display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
def stream_graph_updates(user_input: str):
    for event in graph.stream({"messages": [{"role": "user", "content": user_input}]}):
        print(event)
        for value in event.values():
            print("Assistant:", value["messages"][-1].content)

In [ ]:
while True:
    try:
        user_input = input("User: ")
        if user_input.lower() in ["quit", "exit", "q"]:
            print("Goodbye!")
            break

        stream_graph_updates(user_input)
    except: 
        # fallback if input() is not available
        user_input = "What do you know about LangGraph?"
        print("User: " + user_input)
        stream_graph_updates(user_input)
        break

## LANGRAPH AND CHAINS

In [ ]:
from pprint import pprint
from langchain_core.messages import AIMessage, HumanMessage

messages = [AIMessage(content=f"So you said you were researching ocean mammals?", name="Model")]
messages.append(HumanMessage(content=f"Yes, that's right.",name="Lance"))
messages.append(AIMessage(content=f"Great, what would you like to learn about.", name="Model"))
messages.append(HumanMessage(content=f"I want to learn about the best place to see Orcas in the US.", name="Lance"))

for m in messages:
    m.pretty_print()

In [ ]:
# CREATE CHAIN USING GROQ MODEL FOR CHATBOT AND TOOL

#STEP 1 : IMPORTS and LOAD ENV VARIABLES
import os
import dotenv
from langchain_groq import ChatGroq
from langgraph.graph import StateGraph, START, END
from typing_extensions import TypedDict
from typing import Annotated
from langgraph.graph.message import add_messages
from langchain_core.messages import AnyMessage
from IPython.display import Image, display


dotenv.load_dotenv()
os.environ['GROQ_API_KEY'] = os.environ['GROQ_API_KEY']

In [ ]:
# STEP 2 DEFINE OUR MODEL: 
llm = ChatGroq(model="qwen-2.5-32b")
result = llm.invoke(messages)
print(result.content)

In [ ]:
# STEP 3 : CREATE CUSTOM TOOL TO INTEGRATE
def add(a:int,b:int)->int:
    """ ADD TWO NUMBERS"""
    return a+b

def multiply(a:int,b:int)->int:
    """ MULTIPLIES TWO NUMBERS"""
    return a*b

# STEP  4: BIND THE TOOL WITH LLM
llm_with_tool = llm.bind_tools([add])

# STEP 5: INVOKE THE LLM WITH TOOL FOR EXAMPLE // NOT NEEDED OTHERWISE
tool_call = llm_with_tool.invoke([HumanMessage(content="What is 2+3?", name="Kinnary")])
print(tool_call)
print(tool_call.tool_calls)



In [ ]:
initial_message=[AIMessage(content="Hello! How can I assist you?", name="Model"),
                    HumanMessage(content="I'm looking for information on generative ai.", name="Krish")
                   ]
# New message to add
new_message = AIMessage(content="Sure, I can help with that. What specifically are you interested in?", name="Model")

add_messages(initial_message,new_message)

In [ ]:
# STEP 6: BUILD THE GRAPH


# 6.1 GENERATE MESSAGE STATE CLASS
class MessageState(TypedDict):
    messages:Annotated[list[AnyMessage],add_messages]


# 6.2 CREATE FUNCTION TO INVOKE LLM WITH TOOL WITH CURRENT STATE OF GRAPH
def tool_calling_llm(state: MessageState):
    return {"messages": [llm_with_tool.invoke(state["messages"])]}



# 6.3 CREATE GRAPH BUILDER
graph_builder = StateGraph(MessageState)

# 6.4 ADD NODES
graph_builder.add_node("tool_calling_llm",tool_calling_llm)

# 6.5 ADD EDGES
graph_builder.add_edge(START, "tool_calling_llm")
graph_builder.add_edge("tool_calling_llm", END) 

# 6.6 COMPILE GRAPH
graph = graph_builder.compile()

# 6.7 DISPLAY GRAPH
display(Image(graph.get_graph().draw_mermaid_png()))

messages=graph.invoke({"messages":HumanMessage(content="What is 2 minus 3")})
for m in messages['messages']:
    m.pretty_print()


In [ ]:
from langgraph.prebuilt import ToolNode, tools_condition

llm_with_tool = llm.bind_tools([add, multiply])

# 6.8 ADD TOOL NODE
graph_builder = StateGraph(MessageState)

# 6.8.1 ADD NODES
graph_builder.add_node("llm_with_tool",tool_calling_llm)
graph_builder.add_node("tools",ToolNode([add, multiply]))

# 6.8.2 ADD EDGES
graph_builder.add_edge(START, "llm_with_tool")
graph_builder.add_conditional_edges("llm_with_tool",tools_condition)
graph_builder.add_edge("tools", END)

# 6.8.3 COMPILE GRAPH
graph = graph_builder.compile()     

# 6.8.4 DISPLAY GRAPH
display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
messages= graph.invoke({"messages":HumanMessage(content="What is 2 multiplied 3 plus 4 minus 5")})
for m in messages['messages']:
    m.pretty_print()
